## Objetivo

Este notebook deve unificar os 4 datasets criados a partir dos dados abertos da UFRN. São eles: `Alunos`, `Componentes`, `Matriculas` e `Turmas`.

Infelizmente não é possível relacionar os dados dos Alunos com os das Matriculas, assim sendo, para criar o modelo necessário para predizer se o aluno tem chance de ser aprovado na disciplina, usaremos apenas os 3 últimos datasets listados acima.

### 1. Bibliotecas

In [1]:
library("dplyr")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### 2. Dados

In [2]:
path <- "/home/navi/RDS/HackathonUFRN/resultados"

In [3]:
setwd(path)

componentes <- read.csv2("componentes.csv", header=T, sep=",")
matriculas <- read.csv2("matriculas.csv", header=T, sep=",")
turmas <- read.csv2("turmas.csv", header=T, sep=",")

In [4]:
str(componentes)

'data.frame':	67988 obs. of  8 variables:
 $ id_componente          : int  20001 20001 20001 20002 20003 20003 20003 20003 20003 20003 ...
 $ id_curriculo           : int  104850 104848 104849 125003 107690 107680 107687 107676 107674 107685 ...
 $ semestre_oferta        : int  4 4 4 3 6 9 6 9 7 6 ...
 $ tipo_vinculo_componente: Factor w/ 3 levels "COMPLEMENTAR",..: 2 2 2 2 3 2 3 2 2 3 ...
 $ nivel_ensino           : Factor w/ 1 level "GRADUAÇÃO": 1 1 1 1 1 1 1 1 1 1 ...
 $ nome                   : Factor w/ 10298 levels "A  QUESTÃO DO HUMANO",..: 5710 5710 5710 7685 561 561 561 561 561 561 ...
 $ unidade_responsavel    : Factor w/ 152 levels "CB - COORDENAÇÃO DO CURSO DE CIÊNCIAS BIOLÓGICAS - LICENCIATURA",..: 77 77 77 12 74 74 74 74 74 74 ...
 $ ch_total               : int  60 60 60 90 60 60 60 60 60 60 ...


In [5]:
str(matriculas)

'data.frame':	5747370 obs. of  9 variables:
 $ id_turma           : int  57547303 57547303 57547303 57547303 57547303 57547303 57547303 57547303 57547303 57547303 ...
 $ id_curso           : int  2000072 2000072 2000072 2000072 2000072 2000072 2000072 2000072 2000072 2000072 ...
 $ unidade            : int  1 2 3 1 2 3 1 2 3 1 ...
 $ nota               : Factor w/ 866 levels "0","0.0666667",..: 746 746 746 746 420 420 746 420 420 746 ...
 $ reposicao          : Factor w/ 2 levels "f","t": 1 1 1 1 1 1 1 1 1 1 ...
 $ faltas_unidade     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ media_final        : Factor w/ 101 levels "0.00","0.10",..: 92 92 92 77 77 77 77 77 77 88 ...
 $ numero_total_faltas: int  0 0 0 0 0 0 0 0 0 0 ...
 $ descricao          : Factor w/ 16 levels "APROVADO","APROVADO POR NOTA",..: 1 1 1 1 1 1 1 1 1 1 ...


In [6]:
str(turmas)

'data.frame':	104612 obs. of  6 variables:
 $ id_turma                : int  1151160 1151161 1151162 1151178 1151178 1151179 1151179 1151200 1151200 1151201 ...
 $ id_componente_curricular: int  24054 24061 25843 23857 23857 23857 23857 23667 23667 23646 ...
 $ nivel_ensino            : Factor w/ 1 level "GRADUAÇÃO": 1 1 1 1 1 1 1 1 1 1 ...
 $ periodo                 : int  1 1 1 1 1 1 1 1 1 1 ...
 $ descricao_horario       : Factor w/ 10000 levels "234567M12345  234567T123",..: 4390 9246 4351 1886 1886 1886 1886 9395 9395 9395 ...
 $ tipo                    : Factor w/ 3 levels "ENSINO INDIVIDUAL",..: 2 2 2 2 2 2 2 1 1 1 ...


In [7]:
tmp_1 <- merge(matriculas, turmas, by="id_turma")

In [8]:
str(tmp_1)

'data.frame':	15538888 obs. of  14 variables:
 $ id_turma                : int  1151160 1151160 1151160 1151161 1151161 1151161 1151162 1151162 1151162 1151178 ...
 $ id_curso                : int  2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000056 ...
 $ unidade                 : int  2 3 1 3 1 2 2 1 3 3 ...
 $ nota                    : Factor w/ 866 levels "0","0.0666667",..: 1 1 1 248 581 581 266 420 581 617 ...
 $ reposicao               : Factor w/ 2 levels "f","t": 1 1 1 1 1 1 1 1 1 1 ...
 $ faltas_unidade          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ media_final             : Factor w/ 101 levels "0.00","0.10",..: 1 1 1 72 72 72 72 72 72 81 ...
 $ numero_total_faltas     : int  0 0 0 0 0 0 14 14 14 0 ...
 $ descricao               : Factor w/ 16 levels "APROVADO","APROVADO POR NOTA",..: 10 10 10 1 1 1 1 1 1 1 ...
 $ id_componente_curricular: int  24054 24054 24054 24061 24061 24061 25843 25843 25843 23857 ...
 $ nivel_ensino            : Factor w/ 1 lev

In [9]:
head(tmp_1)

id_turma,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,id_componente_curricular,nivel_ensino,periodo,descricao_horario,tipo
1151160,2000029,2,0,f,0,0.00,0,REPROVADO,24054,GRADUAÇÃO,1,35T12,REGULAR
1151160,2000029,3,0,f,0,0.00,0,REPROVADO,24054,GRADUAÇÃO,1,35T12,REGULAR
1151160,2000029,1,0,f,0,0.00,0,REPROVADO,24054,GRADUAÇÃO,1,35T12,REGULAR
1151161,2000029,3,5.5,f,0,7.00,0,APROVADO,24061,GRADUAÇÃO,1,6T56 5N34,REGULAR
1151161,2000029,1,8,f,0,7.00,0,APROVADO,24061,GRADUAÇÃO,1,6T56 5N34,REGULAR
1151161,2000029,2,8,f,0,7.00,0,APROVADO,24061,GRADUAÇÃO,1,6T56 5N34,REGULAR


In [11]:
lista_nomes <- names(tmp_1)
total <- nrow(tmp_1)

for(i in 1:length(lista_nomes)){
    percent <- ((sum(is.na(tmp_1[,i])))/total)*100
    str <- paste(percent, "% para ", lista_nomes[i], sep="")
    print(str)
}

[1] "0% para id_turma"
[1] "0% para id_curso"
[1] "0% para unidade"
[1] "0% para nota"
[1] "0% para reposicao"
[1] "0% para faltas_unidade"
[1] "0% para media_final"
[1] "0% para numero_total_faltas"
[1] "0% para descricao"
[1] "0% para id_componente_curricular"
[1] "0% para nivel_ensino"
[1] "0% para periodo"
[1] "0% para descricao_horario"
[1] "0% para tipo"


In [12]:
names(tmp_1)[10] <- "id_componente"

str(tmp_1)

'data.frame':	15538888 obs. of  14 variables:
 $ id_turma           : int  1151160 1151160 1151160 1151161 1151161 1151161 1151162 1151162 1151162 1151178 ...
 $ id_curso           : int  2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000029 2000056 ...
 $ unidade            : int  2 3 1 3 1 2 2 1 3 3 ...
 $ nota               : Factor w/ 866 levels "0","0.0666667",..: 1 1 1 248 581 581 266 420 581 617 ...
 $ reposicao          : Factor w/ 2 levels "f","t": 1 1 1 1 1 1 1 1 1 1 ...
 $ faltas_unidade     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ media_final        : Factor w/ 101 levels "0.00","0.10",..: 1 1 1 72 72 72 72 72 72 81 ...
 $ numero_total_faltas: int  0 0 0 0 0 0 14 14 14 0 ...
 $ descricao          : Factor w/ 16 levels "APROVADO","APROVADO POR NOTA",..: 10 10 10 1 1 1 1 1 1 1 ...
 $ id_componente      : int  24054 24054 24054 24061 24061 24061 25843 25843 25843 23857 ...
 $ nivel_ensino       : Factor w/ 1 level "GRADUAÇÃO": 1 1 1 1 1 1 1 1 1 1 ...
 $ periodo     

In [13]:
summary(tmp_1)

    id_turma           id_curso            unidade           nota         
 Min.   : 1151160   Min.   :  2000002   Min.   :1.000   10     :  944229  
 1st Qu.: 1181265   1st Qu.:  2000053   1st Qu.:1.000   9      :  852302  
 Median : 1219928   Median :  2000056   Median :2.000   8      :  775321  
 Mean   :18136622   Mean   : 11763644   Mean   :1.936   8.5    :  679560  
 3rd Qu.:57563255   3rd Qu.:  2000066   3rd Qu.:3.000   0      :  572066  
 Max.   :57612401   Max.   :119512361   Max.   :3.000   7      :  516144  
                                                        (Other):11199266  
 reposicao    faltas_unidade  media_final       numero_total_faltas
 f:15435677   Min.   :0      7.00   :  583301   Min.   :  0.000    
 t:  103211   1st Qu.:0      8.00   :  573587   1st Qu.:  0.000    
              Median :0      8.50   :  541839   Median :  0.000    
              Mean   :0      8.30   :  509971   Mean   :  4.857    
              3rd Qu.:0      8.20   :  500310   3rd Qu.:  6.

In [14]:
summary(componentes)

 id_componente      id_curriculo       semestre_oferta  tipo_vinculo_componente
 Min.   :  20001   Min.   :   100851   Min.   : 0.000   COMPLEMENTAR:   85     
 1st Qu.:  23302   1st Qu.:   119626   1st Qu.: 0.000   OBRIGATÓRIO :34232     
 Median :  25799   Median :   137336   Median : 3.000   OPTATIVO    :33671     
 Mean   : 249747   Mean   : 35583700   Mean   : 3.437                          
 3rd Qu.:  43639   3rd Qu.: 86537522   3rd Qu.: 6.000                          
 Max.   :2056212   Max.   :421273697   Max.   :13.000                          
                                                                               
    nivel_ensino                               nome      
 GRADUAÇÃO:67988   EDUCACAO FISICA I             :  469  
                   EDUCACAO FISICA II            :  464  
                   ESTUDO DE PROB BRASILEIROS I  :  416  
                   ESTUDOS DE PROB BRASILEIROS II:  413  
                   ENADE - INGRESSANTE           :  355  
            

In [15]:
tmp_2 <- merge(tmp_1, componentes, by="id_componente")

In [16]:
str(tmp_2)

'data.frame':	84257331 obs. of  21 variables:
 $ id_componente          : int  20001 20001 20001 20001 20001 20001 20001 20001 20001 20001 ...
 $ id_turma               : int  57550203 57550203 57550203 1178290 1178290 1178290 1208301 1208301 1208301 1175105 ...
 $ id_curso               : int  2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 ...
 $ unidade                : int  1 1 1 1 1 1 2 2 2 1 ...
 $ nota                   : Factor w/ 866 levels "0","0.0666667",..: 746 746 746 64 64 64 64 64 64 64 ...
 $ reposicao              : Factor w/ 2 levels "f","t": 1 1 1 1 1 1 1 1 1 1 ...
 $ faltas_unidade         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ media_final            : Factor w/ 101 levels "0.00","0.10",..: 99 99 99 21 21 21 21 21 21 21 ...
 $ numero_total_faltas    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ descricao              : Factor w/ 16 levels "APROVADO","APROVADO POR NOTA",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ nivel_ensino.x         : Factor w/ 1 level "GRADUAÇÃ

In [18]:
lista_nomes <- names(tmp_2)
total <- nrow(tmp_2)

for(i in 1:length(lista_nomes)){
    percent <- ((sum(is.na(tmp_2[,i])))/total)*100
    str <- paste(percent, "% para ", lista_nomes[i], sep="")
    print(str)
}

[1] "0% para id_componente"
[1] "0% para id_turma"
[1] "0% para id_curso"
[1] "0% para unidade"
[1] "0% para nota"
[1] "0% para reposicao"
[1] "0% para faltas_unidade"
[1] "0% para media_final"
[1] "0% para numero_total_faltas"
[1] "0% para descricao"
[1] "0% para nivel_ensino.x"
[1] "0% para periodo"
[1] "0% para descricao_horario"
[1] "0% para tipo"
[1] "0% para id_curriculo"
[1] "0% para semestre_oferta"
[1] "0% para tipo_vinculo_componente"
[1] "0% para nivel_ensino.y"
[1] "0% para nome"
[1] "0% para unidade_responsavel"
[1] "0% para ch_total"


In [20]:
names(tmp_2)

[1] "id_componente"           "id_turma"               
 [3] "id_curso"                "unidade"                
 [5] "nota"                    "reposicao"              
 [7] "faltas_unidade"          "media_final"            
 [9] "numero_total_faltas"     "descricao"              
[11] "nivel_ensino.x"          "periodo"                
[13] "descricao_horario"       "tipo"                   
[15] "id_curriculo"            "semestre_oferta"        
[17] "tipo_vinculo_componente" "nivel_ensino.y"         
[19] "nome"                    "unidade_responsavel"    
[21] "ch_total"

In [21]:
write.csv(tmp_2, "agrupado_20171122.csv", row.names=FALSE)

In [5]:
setwd(path)

tmp_2 <- read.csv2("agrupado_20171122.csv", header=TRUE, sep=",")

## Gerando o dataset para treinar a RN

In [31]:
reduced <- subset(tmp_2, select=c('id_componente', 'id_curso', 'ch_total', 'descricao_horario',
                                  'periodo', 'descricao'))

In [35]:
str(reduced)

'data.frame':	84257331 obs. of  6 variables:
 $ id_componente: int  20001 20001 20001 20001 20001 20001 20001 20001 20001 20001 ...
 $ id_curso     : int  2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 ...
 $ ch_total     : int  60 60 60 60 60 60 60 60 60 60 ...
 $ horario      : chr  "N" "N" "N" "N" ...
 $ periodo      : int  1 1 1 1 1 1 2 2 2 2 ...
 $ descricao    : Factor w/ 16 levels "APROVADO","APROVADO POR NOTA",..: 1 1 1 1 1 1 1 1 1 1 ...


In [15]:
levels(reduced$descricao)

[1] "APROVADO"                         "APROVADO POR NOTA"               
 [3] "CANCELADO"                        "CUMPRIU"                         
 [5] "DESISTENCIA"                      "DISPENSADO"                      
 [7] "EXCLUIDA"                         "INDEFERIDO"                      
 [9] "MATRICULADO"                      "REPROVADO"                       
[11] "REPROVADO POR FALTAS"             "REPROVADO POR MÉDIA E POR FALTAS"
[13] "REPROVADO POR NOTA"               "REPROVADO POR NOTA E FALTA"      
[15] "TRANCADO"                         "TRANSFERIDO"

In [23]:
setwd(path)

write.csv(reduced, "reduced_20171123.csv", row.names=FALSE)

### normalizando os horarios 

    1 - M
    2 - T
    3 - N
    4 - MT
    5 - MN
    6 - TN
    7 - MTN

In [61]:
setwd(path)

reduced <- read.csv2("reduced_20171123.csv", header=TRUE, sep=",")

In [62]:
names(reduced)[4] <- "horario"

In [63]:
# removendo os numeros e espaços em brancos nos horários

reduced$horario <- gsub(" ", "", gsub('[[:digit:]]+', '', reduced$horario))

In [64]:
# removendo todos os caracteres especiais que existem nos horarios

reduced$horario <- gsub('[[:punct:]]', '', reduced$horario)

In [65]:
# removendo as repetições de turnos dos horários

reduced$horario <- ifelse((grepl("M", reduced$horario) & grepl("T", reduced$horario) & grepl("N", reduced$horario)), 7,
       ifelse((grepl("T", reduced$horario) & grepl("N", reduced$horario)),6,
             ifelse((grepl("M", reduced$horario) & grepl("N", reduced$horario)),5,
                   ifelse((grepl("M", reduced$horario) & grepl("T", reduced$horario)),4,
                         ifelse(grepl("N", reduced$horario), 3,
                               ifelse(grepl("T", reduced$horario), 2,
                                     ifelse(grepl("M", reduced$horario),1,0)))))))

In [66]:
unique(reduced$horario)

[1] 3 2 1 4 6 5 7

### normalizando a descricao

    APROVADO
    REPROVADO
    DESISTENTE
    INDEFERIDO

In [67]:
reduced$descricao <- ifelse(reduced$descricao == 'APROVADO','APROVADO',
                    ifelse(reduced$descricao == 'APROVADO POR NOTA','APROVADO',
                    ifelse(reduced$descricao == 'CUMPRIU','APROVADO',
                    ifelse(reduced$descricao == 'DISPENSADO','APROVADO',
                    ifelse(reduced$descricao == 'REPROVADO','REPROVADO',
                    ifelse(reduced$descricao == 'REPROVADO POR FALTAS','REPROVADO',
                    ifelse(reduced$descricao == 'REPROVADO POR MÉDIA E POR FALTAS','REPROVADO',
                    ifelse(reduced$descricao == 'REPROVADO POR NOTA','REPROVADO',
                    ifelse(reduced$descricao == 'REPROVADO POR NOTA E FALTA','REPROVADO',
                    ifelse(reduced$descricao == 'DESISTENCIA','DESISTENTE',
                    ifelse(reduced$descricao == 'TRANCADO','DESISTENTE',
                    ifelse(reduced$descricao == 'INDEFERIDO','INDEFERIDO','NA'))))))))))))

In [68]:
unique(reduced$descricao)

[1] "APROVADO"   "REPROVADO"  "NA"         "DESISTENTE" "INDEFERIDO"

In [69]:
reduced <- reduced[complete.cases(reduced), ]
reduced <- droplevels(reduced)

str(reduced)

'data.frame':	84257331 obs. of  6 variables:
 $ id_componente: int  20001 20001 20001 20001 20001 20001 20001 20001 20001 20001 ...
 $ id_curso     : int  2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 2000016 ...
 $ ch_total     : int  60 60 60 60 60 60 60 60 60 60 ...
 $ horario      : num  3 3 3 3 3 3 2 2 2 2 ...
 $ periodo      : int  1 1 1 1 1 1 2 2 2 2 ...
 $ descricao    : chr  "APROVADO" "APROVADO" "APROVADO" "APROVADO" ...


In [70]:
setwd(path)

write.csv(reduced, "reduced_20171123_2.csv", row.names=FALSE)

## Gerando os indicadores para os componentes